In [1]:
function house(x)
    m = length(x)
    σ = x[2:m]'x[2:m]
    v = vcat(1, x[2:m])
    if σ == 0 && x[1] >= 0
        β = 0
    elseif σ == 0 && x[1] < 0
        β = -2
    else
        μ = sqrt(x[1]^2 + σ)
        if x[1] <= 0
            v[1] = x[1] - μ
        else
            v[1] = -σ/(x[1] - μ)
        end
        β = 2v[1]^2/(σ + v[1]^2)
        v = v/v[1]
    end
    return v, β
end

house (generic function with 1 method)

In [2]:
function givens(a,b)
    if b == 0
        c = 1
        s = 0
    else
        if abs(b) > abs(a)
            τ = -a/b
            s = 1/sqrt(1 + τ^2)
            c = s*τ
        else
            τ = -b/a
            c = 1/sqrt(1 + τ^2)
            s = c*τ
        end
    end
    return c, s
end

givens (generic function with 1 method)

In [3]:
function houseQR(A)
    m,n = size(A)
    for j in 1:n
        v, β = house(A[j:m,j])
        A[j:m,j:n] = (eye(m-j+1) - β*v*v')*A[j:m,j:n]
        if j < m
            A[j+1:m,j] = v[2:m-j+1]
        end
    end
    return A
end

houseQR (generic function with 1 method)

In [4]:
function getQ(A)
    n,m = size(A)
    Q = eye(m,m)
    for j in n:-1:1
        v = vcat(1, A[j+1:m,j])
        β_j = 2/(1 + sum(A[j+1:m,j].^2))
        Q[j:m,j:m] = Q[j:m,j:m] - β_j*v*v'*Q[j:m,j:m]
    end
    return Q
end

getQ (generic function with 1 method)

In [5]:
function givensQR(A)
    m,n = size(A)
    for j in 1:n
        for i in m:-1:j+1
            c,s = givens(A[i-1,j],A[i,j])
            A[i-1:i,j:n] = [c -s ; s c] * A[i-1:i,j:n]
        end
    end
    return A
end

givensQR (generic function with 1 method)

In [6]:
function cqsQR(A)
    m,n = size(A)
    R = zeros(n,n)
    Q = zeros(m,n)
    R[1,1] = norm(A[:,1],2)
    Q[:,1] = A[:,1]/R[1,1]
    for k in 2:n
        R[1:k-1,k] = Q[1:m,1:k-1]'*A[1:m,k]
        z = A[1:m,k] - Q[1:m,1:k-1]*R[1:k-1,k]
        R[k,k] = norm(z,2)
        Q[1:m,k] = z/R[k,k]
    end
    return Q, R
end

cqsQR (generic function with 1 method)

In [7]:
function mgsQR(A)
    m,n = size(A)
    for k in 1:n
        R[k,k] = norm(A[1:m,k],2)
        Q[1:m,k] = A[1:m,k]/R[k,k]
        for j in k+1:n
            R[k,j] = Q[1:m,k]'*A[1:m,j]
            A[1:m,j] = A[1:m,j] - Q[1:m,k]R[k,j]
        end
    end
    return Q, R
end

mgsQR (generic function with 1 method)